In [21]:
import psycopg2

def execute_sql_script(db_params, sql_file_path):
    # Conectar ao banco de dados
    try:
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()
        
        # Ler o conteúdo do arquivo SQL
        with open(sql_file_path, 'r') as file:
            sql_script = file.read()
        
        # Executar o script SQL
        cursor.execute(sql_script)
        
        # Confirmar as alterações
        conn.commit()
        print("Script SQL executado com sucesso.")
    
    except Exception as e:
        print(f"Erro ao executar o script SQL: {e}")
    
    finally:
        # Fechar a conexão
        if cursor:
            cursor.close()
        if conn:
            conn.close()



In [8]:
import psycopg2
import random
import time
from datetime import datetime
import pytz

# Configurações de conexão com o banco de dados
HOST = "localhost"
PORT = 5432
USER = "admin"
PASSWORD = "admin"
DATABASE = "postgres"  # Substitua pelo nome do seu banco de dados

# Conecta ao banco de dados
conn = psycopg2.connect(
    host=HOST,
    port=PORT,
    user=USER,
    password=PASSWORD,
    dbname=DATABASE
)
cur = conn.cursor()

# Função para gerar e armazenar dados a cada 400 milissegundos
def gerar_e_armazenar_dados():
    tz = pytz.timezone('America/Sao_Paulo')  # Ajuste para o seu fuso horário local
    try:
        while True:
            timestamp = datetime.now().astimezone(tz)  # Converte para o fuso horário local
            temperatura = round(random.uniform(-10, 35), 2)  # Temperatura entre -10 e 35 graus Celsius
            umidade = round(random.uniform(0, 100), 2)  # Umidade entre 0 e 100%
            
            # Insere os dados nas tabelas
            cur.execute("INSERT INTO temperature (time, value) VALUES (%s, %s)", (timestamp, temperatura))
            cur.execute("INSERT INTO moisture (time, value) VALUES (%s, %s)", (timestamp, umidade))
            
            conn.commit()  # Confirma a transação
            
            time.sleep(1)  # Aguarda 400 milissegundos (0.4 segundos)
    except KeyboardInterrupt:
        print("Interrompido pelo usuário.")

# Gera e armazena dados continuamente
gerar_e_armazenar_dados()

# Fecha a conexão (não será alcançado a menos que o loop seja interrompido)
cur.close()
conn.close()


Interrompido pelo usuário.


## MQTT

In [ ]:
import paho.mqtt.client as mqtt
import psycopg2

# Configurações do banco de dados
DB_HOST = 'localhost'
DB_PORT = 5432
DB_NAME = 'postgres'
DB_USER = 'admin'
DB_PASSWORD = 'admin'

# Configurações do MQTT
MQTT_BROKER = 'www.maqiatto.com'
MQTT_PORT = 1883
MQTT_USER = 'lucasl050503@gmail.com'
MQTT_PASSWORD = '123456'

# Funções para o MQTT
def on_connect(client, userdata, flags, rc):
    print("Conectado com sucesso. Código de resultado:", rc)
    client.subscribe("lucasl050503@gmail.com/UPI1/moisture")
    client.subscribe("lucasl050503@gmail.com/UPI1/temperature")

def on_message(client, userdata, msg):
    print(f"Recebido mensagem em {msg.topic}: {msg.payload.decode()}")
    if msg.topic == "lucasl050503@gmail.com/UPI1/moisture":
        insert_data("moisture", msg.payload.decode())
    elif msg.topic == "lucasl050503@gmail.com/UPI1/temperature":
        insert_data("temperature", msg.payload.decode())

def insert_data(sensor_type, value):
    try:
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
        cur = conn.cursor()
        if sensor_type == "moisture":
            cur.execute("INSERT INTO moisture_data (value) VALUES (%s)", (value,))
        elif sensor_type == "temperature":
            cur.execute("INSERT INTO temperature_data (value) VALUES (%s)", (value,))
        conn.commit()
        cur.close()
        conn.close()
    except Exception as e:
        print("Erro ao inserir dados:", e)

# Configuração do cliente MQTT
client = mqtt.Client()
client.username_pw_set(MQTT_USER, MQTT_PASSWORD)
client.on_connect = on_connect
client.on_message = on_message

# Conectar ao broker MQTT
client.connect(MQTT_BROKER, MQTT_PORT, 60)

# Loop para manter o cliente em execução
client.loop_forever()


In [4]:
import paho.mqtt.client as mqtt

MQTT_BROKER = 'www.maqiatto.com'
MQTT_PORT = 1883
MQTT_USER = 'lucasl050503@gmail.com'
MQTT_PASSWORD = '123456'


def on_connect(client, userdata, flags, rc):
    print("Conectado com sucesso. Código de resultado:", rc)
    client.subscribe("lucasl050503@gmail.com/UPI1/moisture")
    client.subscribe("lucasl050503@gmail.com/UPI1/temperature")

def on_message(client, userdata, msg):
    print(f"Recebido mensagem em {msg.topic}: {msg.payload.decode()}")


client = mqtt.Client()
client.username_pw_set(MQTT_USER, MQTT_PASSWORD)
client.on_connect = on_connect
client.on_message = on_message

client.connect(MQTT_BROKER, MQTT_PORT, 60)

client.loop_forever()

C:\Users\Warle\AppData\Local\Temp\ipykernel_16804\2737626424.py:20: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Conectado com sucesso. Código de resultado: 0
Recebido mensagem em lucasl050503@gmail.com/UPI1/moisture: 50.0
Recebido mensagem em lucasl050503@gmail.com/UPI1/temperature: 22.5
Recebido mensagem em lucasl050503@gmail.com/UPI1/moisture: 50.0
Recebido mensagem em lucasl050503@gmail.com/UPI1/temperature: 22.5
Recebido mensagem em lucasl050503@gmail.com/UPI1/moisture: 50.0
Recebido mensagem em lucasl050503@gmail.com/UPI1/temperature: 22.5
Recebido mensagem em lucasl050503@gmail.com/UPI1/moisture: {"machine_id": "machine_123", "sensors": [{"sensor_id": "sensor_1", "data_type": "moisture", "data_interval": 10}]}
Recebido mensagem em lucasl050503@gmail.com/UPI1/temperature: {"machine_id": "machine_123", "sensors": [{"sensor_id": "sensor_2", "data_type": "temperature", "data_interval": 10}]}
Conectado com sucesso. Código de resultado: 0
Conectado com sucesso. Código de resultado: 0


KeyboardInterrupt: 